In [1]:
# Import required modules
import pandas as pd
from google_trans_new import google_translator
from concurrent.futures import ProcessPoolExecutor
import numpy as np
# !pip install ethnicolr
# from  ethnicolr import census_ln, pred_census_ln

In [2]:
# Read in the data to analyse. Unzip "combinedOrdersdata.zip" file
df = pd.read_pickle("combinedOrdersData.pickle")

In [3]:
# Translate text into english
def translateIntoEng(text, column1, column2):
    """text = text to translate,
    column1 = column name for original text,
    column2 = column name for converted text"""
    
    t = google_translator()
    engText = t.translate(text, lang_tgt="en").strip().lower()
    df = pd.DataFrame({
        f"{column1}":text,
        f"{column2}":engText
    }, index=[0])
    
    return df

%%time
with ProcessPoolExecutor() as ex:
    engCityCode = pd.concat(list(map(translateIntoEng,
                                        df.stateCodeBilling.unique(),
                                        ["oldCityCode"]*df.cityBilling.nunique(),
                                        ["engCityCode"]*df.cityBilling.nunique())))

In [4]:
# Create a class to analyse data
class Analyse:
    
    # Calculates distribution of a variable across another variable
    def calIndDist(groupBy, filterBy, toGroup):
        """groupBy = The variable used to group another variable,
        filterBy = any class of 'groupBy' variable,
        toGroup = The variable that will be grouped"""
        
        # Distribution in absolute number of a variable across another variable
        indCount = df[df[f"{groupBy}"]==filterBy][f"{toGroup}"].value_counts().to_frame()\
        .T.reset_index(drop=True)
        indCount[f"{groupBy}"] = filterBy
        
        # Distribution in percentage of a variable across another variable
        indPercentDist = df[df[f"{groupBy}"]==filterBy][f"{toGroup}"].value_counts(normalize=True)\
        .mul(100).round(3).to_frame().T.reset_index(drop=True)
        indPercentDist.columns = indPercentDist.columns + "_%"
        
        # Merge the 2 dfs on column-wise
        mergeInd = pd.concat([indCount, indPercentDist], axis=1)
        return mergeInd
        
    
    # Calculates a variable overall distribution
    def calCombDist(groupBy):
        """groupBy = The variable whose distribution is calculated"""
        
        # # Distribution in absolute number
        combCount = df[f"{groupBy}"].value_counts().to_frame().T.reset_index(drop=True)
        combCount[f"{groupBy}"] = "overall"
        
        # Distribution in %
        combPercentDist = df[f"{groupBy}"].value_counts(normalize=True)\
        .mul(100).round(3).to_frame().T.reset_index(drop=True)
        combPercentDist.columns = combPercentDist.columns + "_%"
        
        # Merge the 2 dfs on column-wise
        mergeComb = pd.concat([combCount, combPercentDist], axis=1)
        return mergeComb

### What is brand-wise and combined male-female students ratio?
#### Takeaway: Overall almost 36% students is male and just over 64% students are female.  John academy has the most female students(72.7%), while TX has the closest male-female students ratio(46.6% male) off all brands.

In [5]:
genderRatioByBrand = pd.concat(list(map(
    Analyse.calIndDist,
    ["courseProvider"]*df.courseProvider.nunique(),
    df.courseProvider.unique(),
    ["gender"]*df.courseProvider.nunique()))).reset_index(drop=True)

genderRatioOverall = Analyse.calCombDist("gender")
display(genderRatioOverall, genderRatioByBrand)

,F,M,gender,F_%,M_%
0,54087,30283,overall,64.107,35.893


,F,M,courseProvider,F_%,M_%
0,6925,5239,course gate,56.930,43.070
1,5753,4500,janets,56.110,43.890
2,7362,4465,alpha academy,62.247,37.753
3,21682,9319,one education,69.940,30.060
4,4299,3737,training express,53.497,46.503
5,8066,3023,john academy,72.739,27.261


### What is brand-wise and combined payment method ratio?
#### Takeaway: Combining all brands, over 51% prefers debit or credit card, almost 43% prefers paypal and rest 6% prefers others. John acdemy students prefer only debit or credit card an paypal. Course gate student has the highest apple pay ratio compared to all brands.

In [6]:
paymentMethodByBrand = pd.concat(
list(map(Analyse.calIndDist,
        ["courseProvider"]*df.courseProvider.nunique(),
        df.courseProvider.unique(),
        ["paymentMethodTitle"]*df.courseProvider.nunique()))).reset_index(drop=True).fillna(0)

paymentMethodOverall = Analyse.calCombDist("paymentMethodTitle")
display(paymentMethodOverall, paymentMethodByBrand)

,debit or credit card,paypal,chrome payment request (stripe),apple pay (stripe),cash on delivery,paymentMethodTitle,debit or credit card_%,paypal_%,chrome payment request (stripe)_%,apple pay (stripe)_%,cash on delivery_%
0,41769,35094,2763,2044,3,overall,51.142,42.969,3.383,2.503,0.004


,debit or credit card,paypal,apple pay (stripe),chrome payment request (stripe),courseProvider,debit or credit card_%,paypal_%,apple pay (stripe)_%,chrome payment request (stripe)_%,cash on delivery,cash on delivery_%
0,5778,5198,531,519,course gate,48.046,43.223,4.415,4.316,0,0.00
1,5142,3879,388,399,janets,52.427,39.549,3.956,4.068,0,0.00
2,5845,5045,122,481,alpha academy,50.857,43.896,1.062,4.185,0,0.00
3,14686,13467,820,1108,one education,48.817,44.765,2.726,3.683,3,0.01
4,4205,2661,183,256,training express,57.563,36.427,2.505,3.504,0,0.00
5,6113,4844,0,0,john academy,55.791,44.209,0.000,0.000,0,0.00


### Which country cash on delivery was made from?
#### Takeaway: All the cash on delivery was made Bangladesh

In [7]:
df[df.paymentMethodTitle=="cash on delivery"].countryCodeBilling.unique()

<StringArray>
['bd']
Length: 1, dtype: string

### What is gender-wise payment method?
#### Takeaway: Male students tend to use debit or credit card more than female(52.3%/50.5%), while female student tend to use paypal more than male(38.9%/45%).  Over 6% male student use chrome payment requests, while it is only 1.8% for female.

In [8]:
paymentMethodByGender = pd.concat(
list(map(Analyse.calIndDist,
         ["gender"]*df.paymentMethodTitle.nunique(),
        df.gender.unique(),
        ["paymentMethodTitle"]*df.paymentMethodTitle.nunique()))).reset_index(drop=True).fillna(0)
display(paymentMethodByGender)

,debit or credit card,paypal,chrome payment request (stripe),apple pay (stripe),cash on delivery,gender,debit or credit card_%,paypal_%,chrome payment request (stripe)_%,apple pay (stripe)_%,cash on delivery_%
0,15111,10893,1787,694,1,M,53.047,38.240,6.273,2.436,0.004
1,26658,24201,976,1350,2,F,50.121,45.502,1.835,2.538,0.004


### What is brand-wise students ratio by a country and top 10 countries by student?
#### Takeaway: Over 40.5% GB's students are from One Education, while it is only 2% for TX. Over 46% US's students are from John academy, while it is only 0% for TX. For Saudi Arabia(54,7%), Bahrain(41.5), Qatar(57.4), and UAE(52.4), its Alpha Academy that has the most students by a brand. For T&T, its Janets(31.8) who got the most share by a brand.

#### Undoubtedly, It GB from where our most of the students are (88%). Almost 1% students are from UAE, over 0.8% are from South Africa, 0.78% are from Saudi Arabia, and 0.74% are from Ireland.

In [9]:
studentByCountry = pd.concat(
list(map(Analyse.calIndDist,
        ["countryCodeBilling"]*df.countryCodeBilling.nunique(),
        df.countryCodeBilling.unique(),
        ["courseProvider"]*df.countryCodeBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)


# Top 10 countries by student
studentCountByCountry = df.countryCodeBilling.value_counts().to_frame("count")\
.reset_index().rename(columns={"index":"country"}).head(10)
studentPercentByCountry = df.countryCodeBilling.value_counts(normalize=True).mul(100)\
.round(2).to_frame("percent").reset_index().rename(columns={"index":"country"}).head(10)

overAllStudentByCountry = pd.merge(studentCountByCountry, studentPercentByCountry, on="country")
display(studentByCountry.head(20), overAllStudentByCountry)

,one education,course gate,alpha academy,janets,john academy,training express,countryCodeBilling,one education_%,course gate_%,alpha academy_%,janets_%,john academy_%,training express_%
0,27877,11280,9584,9236,8986,1416,gb,40.768,16.496,14.016,13.507,13.141,2.071
1,233,66,167,45,61,3,ie,40.522,11.478,29.043,7.826,10.609,0.522
2,61,2,20,23,48,2,fr,39.103,1.282,12.821,14.744,30.769,1.282
3,47,32,79,41,172,4,ng,12.533,8.533,21.067,10.933,45.867,1.067
4,4,0,5,4,1,0,vn,28.571,0.000,35.714,28.571,7.143,0.000
5,5,3,23,9,13,0,sg,9.434,5.660,43.396,16.981,24.528,0.000
6,138,19,393,26,170,3,ae,18.425,2.537,52.470,3.471,22.697,0.401
7,124,60,55,34,236,0,us,24.361,11.788,10.806,6.680,46.365,0.000
8,5,5,4,5,11,1,eg,16.129,16.129,12.903,16.129,35.484,3.226
9,62,38,15,18,61,0,it,31.959,19.588,7.732,9.278,31.443,0.000


,country,count,percent
0,gb,68379,87.95
1,bd,1220,1.57
2,ae,749,0.96
3,za,649,0.83
4,sa,608,0.78
5,ie,575,0.74
6,us,509,0.65
7,ng,375,0.48
8,qa,249,0.32
9,es,211,0.27


### What is country-wise male-female ratio?
#### Takeaway: For GB and US, Qatar, UAE and Ireland female students ratio outnumbers corresponding male students ratio. For Saudi Arabia, Bahrain, and Canada, male student ratio is higher than those female students ratio. The ratio is exaxctly 50%-50% for Sweden.

In [10]:
genderByCountry = pd.concat(
list(map(Analyse.calIndDist,
        ["countryCodeBilling"]*df.countryCodeBilling.nunique(),
        df.countryCodeBilling.unique(),
        ["gender"]*df.countryCodeBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(genderByCountry.head(10))

,F,M,countryCodeBilling,F_%,M_%
0,45830.0,22549.0,gb,67.024,32.976
1,370.0,205.0,ie,64.348,35.652
2,89.0,67.0,fr,57.051,42.949
3,241.0,134.0,ng,64.267,35.733
4,4.0,10.0,vn,28.571,71.429
5,27.0,26.0,sg,50.943,49.057
6,366.0,383.0,ae,48.865,51.135
7,317.0,192.0,us,62.279,37.721
8,19.0,12.0,eg,61.290,38.710
9,104.0,90.0,it,53.608,46.392


### What is country-wise payment method ratio?
#### Takeaway: For most of the countries, debit or credit card is the favourite payment method system. Students from countries like Netherlands, Denmark, and China prefer paypal as their favourite payment method compared to other payment system.

In [11]:
paymentMethodByCountry = pd.concat(
list(map(Analyse.calIndDist,
        ["countryCodeBilling"]*df.countryCodeBilling.nunique(),
        df.countryCodeBilling.unique(),
        ["paymentMethodTitle"]*df.countryCodeBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(paymentMethodByCountry.head(20))

,debit or credit card,paypal,chrome payment request (stripe),apple pay (stripe),countryCodeBilling,debit or credit card_%,paypal_%,chrome payment request (stripe)_%,apple pay (stripe)_%,cash on delivery,cash on delivery_%
0,32985,30601,2394,1872,gb,48.613,45.100,3.528,2.759,0,0.000
1,297,239,31,3,ie,52.105,41.930,5.439,0.526,0,0.000
2,78,72,1,0,fr,51.656,47.682,0.662,0.000,0,0.000
3,300,66,6,0,ng,80.645,17.742,1.613,0.000,0,0.000
4,10,4,0,0,vn,71.429,28.571,0.000,0.000,0,0.000
5,20,31,2,0,sg,37.736,58.491,3.774,0.000,0,0.000
6,651,54,5,16,ae,89.669,7.438,0.689,2.204,0,0.000
7,252,192,10,0,us,55.507,42.291,2.203,0.000,0,0.000
8,19,5,6,0,eg,63.333,16.667,20.000,0.000,0,0.000
9,81,108,3,0,it,42.188,56.250,1.562,0.000,0,0.000


### What is country-wise and overall order status?
#### Takeaway: Overall over 99% order status are complete, almost 0.5% is pending, 0.41% is in processing and 0.046% is on hold.
#### Bahrain, Sweden, Thailand and Canada have no other status except completed. Ireland has the most (0.87%) on hold order status followed by US.

In [12]:
orderStatusByCountry = pd.concat(
list(map(Analyse.calIndDist,
        ["countryCodeBilling"]*df.countryCodeBilling.nunique(),
        df.countryCodeBilling.unique(),
        ["orderStatus"]*df.countryCodeBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
orderStatusOverall = Analyse.calCombDist("orderStatus")
display(orderStatusOverall, orderStatusByCountry.head(10))

,completed,pending payment,processing,on hold,orderStatus,completed_%,pending payment_%,processing_%,on hold_%
0,83569,413,349,39,overall,99.051,0.49,0.414,0.046


,completed,pending payment,processing,on hold,countryCodeBilling,completed_%,pending payment_%,processing_%,on hold_%
0,67660,370,319,30,gb,98.949,0.541,0.467,0.044
1,570,0,0,5,ie,99.130,0.000,0.000,0.870
2,150,0,6,0,fr,96.154,0.000,3.846,0.000
3,368,4,3,0,ng,98.133,1.067,0.800,0.000
4,11,0,3,0,vn,78.571,0.000,21.429,0.000
5,53,0,0,0,sg,100.000,0.000,0.000,0.000
6,746,3,0,0,ae,99.599,0.401,0.000,0.000
7,505,0,2,2,us,99.214,0.000,0.393,0.393
8,31,0,0,0,eg,100.000,0.000,0.000,0.000
9,192,2,0,0,it,98.969,1.031,0.000,0.000


### What is brand-wise order status?
#### Takeaway: Janets has the most (3%) "processing" order status. One education has the most (1.12) "pending" order status. John and Alpha academy have all the orders completed.

In [13]:
orderStatusByBrand = pd.concat(
list(map(Analyse.calIndDist,
        ["courseProvider"]*df.courseProvider.nunique(),
        df.courseProvider.unique(),
        ["orderStatus"]*df.courseProvider.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(orderStatusByBrand)

,completed,on hold,courseProvider,completed_%,on hold_%,processing,pending payment,processing_%,pending payment_%
0,12143,21,course gate,99.827,0.173,0,0,0.000,0.000
1,9881,9,janets,96.372,0.088,316,47,3.082,0.458
2,11827,0,alpha academy,100.000,0.000,0,0,0.000,0.000
3,30595,7,one education,98.690,0.023,33,366,0.106,1.181
4,8034,2,training express,99.975,0.025,0,0,0.000,0.000
5,11089,0,john academy,100.000,0.000,0,0,0.000,0.000


### What is gender-wise order status?
#### Takeaway: Female students have almost 90% order status "on hold" compared to male students.

In [14]:
orderStatusByGender = pd.concat(
list(map(Analyse.calIndDist,
        ["orderStatus"]*df.orderStatus.nunique(),
        df.orderStatus.unique(),
        ["gender"]*df.orderStatus.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(orderStatusByGender)

,F,M,orderStatus,F_%,M_%
0,53554,30015,completed,64.084,35.916
1,34,5,on hold,87.179,12.821
2,207,142,processing,59.312,40.688
3,292,121,pending payment,70.702,29.298


### What is state-wise male-female ratio?
#### Takeaway: For every state, male students are outnumbered by female students except for Chesire, Newton mearns, and KSA.

In [15]:
genderByState = pd.concat(
list(map(Analyse.calIndDist,
        ["stateCodeBilling"]*df.stateCodeBilling.nunique(),
        df.stateCodeBilling.unique(),
        ["gender"]*df.stateCodeBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(genderByState.head(20))

,stateCodeBilling,F,M,F_%,M_%
0,oxfordshire,192.0,116.0,62.338,37.662
1,west sussex,363.0,122.0,74.845,25.155
2,nottinghamshire,256.0,65.0,79.751,20.249
3,cheshire,479.0,200.0,70.545,29.455
4,derbyshire,335.0,118.0,73.951,26.049
5,kent,1219.0,490.0,71.328,28.672
6,wales,88.0,37.0,70.400,29.600
7,norfolk,327.0,168.0,66.061,33.939
8,dorset,344.0,102.0,77.130,22.870
9,co,76.0,24.0,76.000,24.000


### What is state-wise payment method?
#### Takeaway: England students mostly pay through apple pay than any other payment methods. Highland, West sussex, Cheshire, and Leicestershire students use paypal more than any other payment methods.

In [16]:
paymentMethodByState = pd.concat(
list(map(Analyse.calIndDist,
        ["stateCodeBilling"]*df.stateCodeBilling.nunique(),
        df.stateCodeBilling.unique(),
        ["paymentMethodTitle"]*df.stateCodeBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(paymentMethodByState.head(20))

,stateCodeBilling,debit or credit card,paypal,chrome payment request (stripe),apple pay (stripe),debit or credit card_%,paypal_%,chrome payment request (stripe)_%,apple pay (stripe)_%,cash on delivery,cash on delivery_%
0,oxfordshire,164,127,10,4,53.770,41.639,3.279,1.311,0,0.0
1,west sussex,219,247,10,6,45.436,51.245,2.075,1.245,0,0.0
2,nottinghamshire,152,155,6,5,47.799,48.742,1.887,1.572,0,0.0
3,cheshire,320,322,14,22,47.198,47.493,2.065,3.245,0,0.0
4,derbyshire,209,227,8,8,46.239,50.221,1.770,1.770,0,0.0
5,kent,917,735,33,22,53.720,43.058,1.933,1.289,0,0.0
6,wales,38,50,1,36,30.400,40.000,0.800,28.800,0,0.0
7,norfolk,211,271,3,6,42.974,55.193,0.611,1.222,0,0.0
8,dorset,187,252,1,4,42.117,56.757,0.225,0.901,0,0.0
9,co,33,26,2,0,54.098,42.623,3.279,0.000,0,0.0


### what is state-wise order status?
#### Takeaway: Students from Kent has the highest pending payment ratio than any other state. Students from Staffordshire has the highest "on hold" order status. Students from Leicestershire has the highest "processing" order status. There is no order status except completed for states Highland, Co durham, and Chesire.

In [17]:
orderStatusByState = pd.concat(
list(map(Analyse.calIndDist,
        ["stateCodeBilling"]*df.stateCodeBilling.nunique(),
        df.stateCodeBilling.unique(),
        ["orderStatus"]*df.stateCodeBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(orderStatusByState.head(10))

,stateCodeBilling,completed,processing,pending payment,completed_%,processing_%,pending payment_%,on hold,on hold_%
0,oxfordshire,302,4,2,98.052,1.299,0.649,0,0.000
1,west sussex,483,0,2,99.588,0.000,0.412,0,0.000
2,nottinghamshire,319,1,1,99.377,0.312,0.312,0,0.000
3,cheshire,669,10,0,98.527,1.473,0.000,0,0.000
4,derbyshire,453,0,0,100.000,0.000,0.000,0,0.000
5,kent,1672,18,17,97.835,1.053,0.995,2,0.117
6,wales,123,0,2,98.400,0.000,1.600,0,0.000
7,norfolk,490,1,4,98.990,0.202,0.808,0,0.000
8,dorset,443,1,2,99.327,0.224,0.448,0,0.000
9,co,95,0,0,95.000,0.000,0.000,5,5.000


### What is state-wise course provider?
#### Takeaway: Kent, Highland, Englang, West sussex's most of the students are from One education. Most TX courses are sold in Northants while most John academy courses are sold in West sussesx.

In [18]:
courseProviderByState = pd.concat(
list(map(Analyse.calIndDist,
        ["stateCodeBilling"]*df.stateCodeBilling.nunique(),
        df.stateCodeBilling.unique(),
        ["courseProvider"]*df.stateCodeBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(courseProviderByState.head(15))

,stateCodeBilling,one education,janets,alpha academy,john academy,course gate,one education_%,janets_%,alpha academy_%,john academy_%,course gate_%,training express,training express_%
0,oxfordshire,110,93,41,39,25,35.714,30.195,13.312,12.662,8.117,0,0.000
1,west sussex,155,43,107,116,61,31.959,8.866,22.062,23.918,12.577,3,0.619
2,nottinghamshire,134,45,42,58,36,41.745,14.019,13.084,18.069,11.215,6,1.869
3,cheshire,257,62,90,90,154,37.850,9.131,13.255,13.255,22.680,26,3.829
4,derbyshire,187,55,62,67,75,41.280,12.141,13.687,14.790,16.556,7,1.545
5,kent,716,268,246,202,254,41.896,15.682,14.394,11.820,14.862,23,1.346
6,wales,56,15,20,6,20,44.800,12.000,16.000,4.800,16.000,8,6.400
7,norfolk,226,53,59,70,73,45.657,10.707,11.919,14.141,14.747,14,2.828
8,dorset,178,46,75,55,77,39.910,10.314,16.816,12.332,17.265,15,3.363
9,co,56,10,18,5,11,56.000,10.000,18.000,5.000,11.000,0,0.000


### What is city-wise gender ratio?
#### Takeaway: Bishop auckland (m/f=60%/40%) and Northallerton (58%42%) have higher male than female students. For most of the city, female students ratio is higher than male.

In [19]:
%%time
with ProcessPoolExecutor() as ex:
    genderByCity = pd.concat(
        list(ex.map(Analyse.calIndDist,
                ["cityBilling"]*df.cityBilling.nunique(),
                df.cityBilling.unique(),
                ["gender"]*df.cityBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(genderByCity.head(15))

,cityBilling,F,M,F_%,M_%
0,didcot,26.0,4.0,86.667,13.333
1,northampton,217.0,98.0,68.889,31.111
2,redcar,13.0,12.0,52.000,48.000
3,billingshurst,11.0,1.0,91.667,8.333
4,blidworth,0.0,1.0,0.000,100.000
5,warrington,95.0,60.0,61.290,38.710
6,belper,22.0,12.0,64.706,35.294
7,tunbridge wells,48.0,17.0,73.846,26.154
8,manchester,616.0,358.0,63.244,36.756
9,cardiff,179.0,123.0,59.272,40.728


CPU times: user 20.7 s, sys: 1.01 s, total: 21.7 s
Wall time: 36.2 s


### What is provider ratio by city?
#### Takeaway: One education, as expected, is the best provider for most of the cities except Bognor regis where its Course gate with most of the students.

In [20]:
%%time
with ProcessPoolExecutor() as ex:
    providerByCity = pd.concat(
        list(ex.map(Analyse.calIndDist,
                ["cityBilling"]*df.cityBilling.nunique(),
                df.cityBilling.unique(),
                ["courseProvider"]*df.cityBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(providerByCity.head(15))

,cityBilling,one education,course gate,janets,john academy,alpha academy,training express,one education_%,course gate_%,janets_%,john academy_%,alpha academy_%,training express_%
0,didcot,13,7,3,3,2,2,43.333,23.333,10.000,10.000,6.667,6.667
1,northampton,140,62,28,49,31,5,44.444,19.683,8.889,15.556,9.841,1.587
2,redcar,7,2,1,6,9,0,28.000,8.000,4.000,24.000,36.000,0.000
3,billingshurst,2,0,0,1,9,0,16.667,0.000,0.000,8.333,75.000,0.000
4,blidworth,0,0,0,0,0,1,0.000,0.000,0.000,0.000,0.000,100.000
5,warrington,55,37,13,24,23,3,35.484,23.871,8.387,15.484,14.839,1.935
6,belper,18,0,6,6,3,1,52.941,0.000,17.647,17.647,8.824,2.941
7,tunbridge wells,22,10,5,15,11,2,33.846,15.385,7.692,23.077,16.923,3.077
8,manchester,370,173,140,119,158,14,37.988,17.762,14.374,12.218,16.222,1.437
9,cardiff,147,54,31,40,24,6,48.675,17.881,10.265,13.245,7.947,1.987


CPU times: user 24.9 s, sys: 1.09 s, total: 26 s
Wall time: 40.4 s


### What is city-wise order status?
#### Takeaway: City "Manchester" has the highest pending (1.02%) order status. City "Stockport" has the highest processing (3.44%)  order status.

In [21]:
%%time
with ProcessPoolExecutor() as ex:
    orderStatusByCity = pd.concat(
        list(ex.map(Analyse.calIndDist,
                ["cityBilling"]*df.cityBilling.nunique(),
                df.cityBilling.unique(),
                ["orderStatus"]*df.cityBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(orderStatusByCity.head(15))

,cityBilling,completed,completed_%,processing,processing_%,pending payment,pending payment_%,on hold,on hold_%
0,didcot,30,100.000,0,0.000,0,0.000,0,0.000
1,northampton,314,99.683,1,0.317,0,0.000,0,0.000
2,redcar,25,100.000,0,0.000,0,0.000,0,0.000
3,billingshurst,12,100.000,0,0.000,0,0.000,0,0.000
4,blidworth,1,100.000,0,0.000,0,0.000,0,0.000
5,warrington,153,98.710,2,1.290,0,0.000,0,0.000
6,belper,34,100.000,0,0.000,0,0.000,0,0.000
7,tunbridge wells,64,98.462,0,0.000,1,1.538,0,0.000
8,manchester,964,98.973,0,0.000,10,1.027,0,0.000
9,cardiff,295,97.682,4,1.325,0,0.000,3,0.993


CPU times: user 22.8 s, sys: 1.15 s, total: 23.9 s
Wall time: 38.7 s


### What is city-wise payment method status?
#### Takeaway: Overall payment method ratio is in favour of debit or credit card. However, students from Leicester use paypal(49.4%) more than debit or credit card (47.4%). The same is also tru for city Bliston, Inverness, Bognor regis, Keith, and Northallerton. Students from Bishop auckland use paypal 24.4%  as their payment method.

In [22]:
%%time
with ProcessPoolExecutor() as ex:
    paymentByCity = pd.concat(
        list(ex.map(Analyse.calIndDist,
                ["cityBilling"]*df.cityBilling.nunique(),
                df.cityBilling.unique(),
                ["paymentMethodTitle"]*df.cityBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(paymentByCity.head(15))

,cityBilling,debit or credit card,paypal,apple pay (stripe),debit or credit card_%,paypal_%,apple pay (stripe)_%,chrome payment request (stripe),chrome payment request (stripe)_%,cash on delivery,cash on delivery_%
0,didcot,15,13,2,50.000,43.333,6.667,0,0.000,0,0.0
1,northampton,183,116,7,58.842,37.299,2.251,5,1.608,0,0.0
2,redcar,8,17,0,32.000,68.000,0.000,0,0.000,0,0.0
3,billingshurst,11,1,0,91.667,8.333,0.000,0,0.000,0,0.0
4,blidworth,1,0,0,100.000,0.000,0.000,0,0.000,0,0.0
5,warrington,81,59,4,52.597,38.312,2.597,10,6.494,0,0.0
6,belper,25,8,1,73.529,23.529,2.941,0,0.000,0,0.0
7,tunbridge wells,37,26,0,56.923,40.000,0.000,2,3.077,0,0.0
8,manchester,473,445,18,48.713,45.829,1.854,35,3.605,0,0.0
9,cardiff,128,159,9,42.384,52.649,2.980,6,1.987,0,0.0


CPU times: user 24.1 s, sys: 1.19 s, total: 25.3 s
Wall time: 39.4 s


In [23]:
# Convert order date into pandas datetime format
df.orderDate = pd.to_datetime(df.orderDate)
df["date"] = df.orderDate.dt.date
df["day"] = df.orderDate.dt.day_name()
df["time"] = df.orderDate.dt.time
df["hour"] = df.orderDate.dt.hour
df["minute"] = df.orderDate.dt.minute
df["second"] = df.orderDate.dt.second

In [24]:
# Create decimal time combining hour and minute
df["time"] = df["hour"].astype("str") + "." + df["minute"].astype("str")
df.time = df.time.astype("float")

In [25]:
def createTimeQuarter(x):
    if 0<=x<=5.99:
        return "12_am_6_am"
    
    if 6<=x<=11.99:
        return "6_am_12_pm"
    
    if 12<=x<=17.99:
        return "12_pm_6_pm"
    
    if 18<=x<=23.99:
        return "6_pm_12_am"
    
    else:
        return "na"

# Apply the function and create time category
df["time"] = df.time.apply(createTimeQuarter)

### What is brand-wise time quarter distribution?
#### Takeaway: Overall most of the courses (35.4%) were purchased between 12 pm to 6 pm, followed by 25% for 6 pm to 12 am.
#### Best quarter for all brands is always 12 pm to 6 pm. 2nd best quarter for One Education is 6 pm to 12 am, while for TX its 6 am to 12 pm.

In [26]:
providerByTimeQuarter =  pd.concat(
    list(map(Analyse.calIndDist,
            ["courseProvider"]*df.courseProvider.nunique(),
            df.courseProvider.unique(),
            ["time"]*df.courseProvider.nunique()))).reset_index(drop=True)

overAllTimeQuarter = Analyse.calCombDist("time")
display(overAllTimeQuarter, providerByTimeQuarter)

,12_pm_6_pm,6_pm_12_am,6_am_12_pm,12_am_6_am,time,12_pm_6_pm_%,6_pm_12_am_%,6_am_12_pm_%,12_am_6_am_%
0,29836,21135,17384,16015,overall,35.363,25.05,20.604,18.982


,12_pm_6_pm,6_pm_12_am,6_am_12_pm,12_am_6_am,courseProvider,12_pm_6_pm_%,6_pm_12_am_%,6_am_12_pm_%,12_am_6_am_%
0,5155.0,3381.0,3060.0,568,course gate,42.379,27.795,25.156,4.670
1,4352.0,2994.0,2290.0,617,janets,42.446,29.201,22.335,6.018
2,NaN,NaN,NaN,11827,alpha academy,NaN,NaN,NaN,100.000
3,12494.0,9789.0,6718.0,2000,one education,40.302,31.576,21.670,6.451
4,3297.0,1823.0,2557.0,359,training express,41.028,22.685,31.819,4.467
5,4538.0,3148.0,2759.0,644,john academy,40.923,28.388,24.881,5.808


### What is gender-wise time quarter distribution?
#### Takeaway: 65.4% students were female when the course were purchased between 12 am to 6 am.

In [27]:
timeQuarterByGender = pd.concat(
    list(map(Analyse.calIndDist,
            ["time"]*df.time.nunique(),
            df.time.unique(),
            ["gender"]*df.time.nunique()))).reset_index(drop=True)

display(timeQuarterByGender)

,F,M,time,F_%,M_%
0,13771,7364,6_pm_12_am,65.157,34.843
1,11027,6357,6_am_12_pm,63.432,36.568
2,9804,6211,12_am_6_am,61.218,38.782
3,19485,10351,12_pm_6_pm,65.307,34.693


### What is payment-method wise time quarter distribution?
#### Takeaway: Most of the payments methods were most used in between 12 pm to 6 pm.

In [28]:
timeQuarterByPaymentMethod = pd.concat(
    list(map(Analyse.calIndDist,
            ["paymentMethodTitle"]*df.paymentMethodTitle.nunique(),
            df.paymentMethodTitle.unique(),
            ["time"]*df.paymentMethodTitle.nunique()))).dropna(how="all").reset_index(drop=True)

display(timeQuarterByPaymentMethod)

,12_pm_6_pm,6_pm_12_am,6_am_12_pm,12_am_6_am,paymentMethodTitle,12_pm_6_pm_%,6_pm_12_am_%,6_am_12_pm_%,12_am_6_am_%
0,15174.0,10032.0,8773.0,7790.0,debit or credit card,36.328,24.018,21.004,18.650
1,12300.0,9457.0,6683.0,6654.0,paypal,35.049,26.948,19.043,18.961
2,820.0,644.0,350.0,230.0,apple pay (stripe),40.117,31.507,17.123,11.252
3,993.0,699.0,480.0,591.0,chrome payment request (stripe),35.939,25.299,17.372,21.390


#### What is order status-wise time quarter distribution?

In [29]:
timeQuarterByOrderStatus = pd.concat(
    list(map(Analyse.calIndDist,
            ["orderStatus"]*df.orderStatus.nunique(),
            df.orderStatus.unique(),
            ["time"]*df.orderStatus.nunique()))).dropna(how="all").reset_index(drop=True)

display(timeQuarterByOrderStatus)

,12_pm_6_pm,6_pm_12_am,6_am_12_pm,12_am_6_am,orderStatus,12_pm_6_pm_%,6_pm_12_am_%,6_am_12_pm_%,12_am_6_am_%
0,29589,20855,17186,15939,completed,35.407,24.955,20.565,19.073
1,22,5,8,4,on hold,56.410,12.821,20.513,10.256
2,100,126,91,32,processing,28.653,36.103,26.074,9.169
3,125,149,99,40,pending payment,30.266,36.077,23.971,9.685


#### What is brand-wise day distribution?

In [30]:
dayByProvider = pd.concat(
    list(map(Analyse.calIndDist,
            ["courseProvider"]*df.day.nunique(),
            df.courseProvider.unique(),
            ["day"]*df.day.nunique()))).reset_index(drop=True)
display(dayByProvider)

,Tuesday,Monday,Friday,Thursday,Wednesday,Saturday,Sunday,courseProvider,Tuesday_%,Monday_%,Friday_%,Thursday_%,Wednesday_%,Saturday_%,Sunday_%
0,2262,2072,1925,1904,1878,1141,982,course gate,18.596,17.034,15.825,15.653,15.439,9.380,8.073
1,1585,1555,1649,1328,1474,1226,1436,janets,15.459,15.166,16.083,12.952,14.376,11.957,14.006
2,1843,1858,1611,1757,1716,1541,1501,alpha academy,15.583,15.710,13.621,14.856,14.509,13.030,12.691
3,4761,5196,5202,4812,4692,3240,3098,one education,15.358,16.761,16.780,15.522,15.135,10.451,9.993
4,1300,1284,1329,1455,1195,718,755,training express,16.177,15.978,16.538,18.106,14.871,8.935,9.395
5,1707,1767,1582,1730,1713,1248,1342,john academy,15.394,15.935,14.266,15.601,15.448,11.254,12.102


#### What is payment method-wise day distribution?

In [31]:
dayByPaymentMethod = pd.concat(
    list(map(Analyse.calIndDist,
            ["paymentMethodTitle"]*df.day.nunique(),
            df.paymentMethodTitle.unique(),
            ["day"]*df.day.nunique()))).reset_index(drop=True).dropna(how="all").fillna(0)
display(dayByPaymentMethod)

,Monday,Tuesday,Friday,Thursday,Wednesday,Sunday,Saturday,paymentMethodTitle,Monday_%,Tuesday_%,Friday_%,Thursday_%,Wednesday_%,Sunday_%,Saturday_%
0,6819.0,6774.0,6583.0,6414.0,6240.0,4528.0,4411.0,debit or credit card,16.326,16.218,15.760,15.356,14.939,10.841,10.560
1,5739.0,5483.0,5457.0,5270.0,5274.0,3866.0,4005.0,paypal,16.353,15.624,15.550,15.017,15.028,11.016,11.412
2,348.0,306.0,332.0,309.0,303.0,236.0,210.0,apple pay (stripe),17.025,14.971,16.243,15.117,14.824,11.546,10.274
4,426.0,456.0,453.0,442.0,420.0,273.0,293.0,chrome payment request (stripe),15.418,16.504,16.395,15.997,15.201,9.881,10.604
5,0.0,3.0,0.0,0.0,0.0,0.0,0.0,cash on delivery,0.000,100.000,0.000,0.000,0.000,0.000,0.000


####  What is order status-wise day distribution?

In [32]:
dayByOrderStatus = pd.concat(
    list(map(Analyse.calIndDist,
            ["orderStatus"]*df.day.nunique(),
            df.orderStatus.unique(),
            ["day"]*df.day.nunique()))).reset_index(drop=True).dropna(how="all").fillna(0)
display(dayByOrderStatus)

,Monday,Tuesday,Friday,Thursday,Wednesday,Sunday,Saturday,orderStatus,Monday_%,Tuesday_%,Friday_%,Thursday_%,Wednesday_%,Sunday_%,Saturday_%
0,13601,13323,13168,12870,12556,9052,8999,completed,16.275,15.943,15.757,15.400,15.025,10.832,10.768
1,2,8,12,4,8,4,1,on hold,5.128,20.513,30.769,10.256,20.513,10.256,2.564
2,52,51,58,59,53,31,45,processing,14.900,14.613,16.619,16.905,15.186,8.883,12.894
3,77,76,60,53,51,27,69,pending payment,18.644,18.402,14.528,12.833,12.349,6.538,16.707


#### What is gender-wise day distribution?

In [33]:
dayByGender = pd.concat(
    list(map(Analyse.calIndDist,
            ["gender"]*df.day.nunique(),
            df.gender.unique(),
            ["day"]*df.day.nunique()))).reset_index(drop=True).dropna(how="all").fillna(0)
display(dayByGender)

,Monday,Thursday,Tuesday,Friday,Wednesday,Saturday,Sunday,gender,Monday_%,Thursday_%,Tuesday_%,Friday_%,Wednesday_%,Saturday_%,Sunday_%
0,4954,4868,4817,4757,4295,3311,3281,M,16.359,16.075,15.907,15.708,14.183,10.934,10.834
1,8778,8118,8641,8541,8373,5803,5833,F,16.229,15.009,15.976,15.791,15.481,10.729,10.784


#### Insert ethnicity
df = census_ln(df, "lastNameBilling")

df[["lastNameBilling", "pctwhite", "pctblack", "pctapi", "pctaian", "pct2prace", "pcthispanic"]]\
.apply(lambda x: pd.to_numeric(x, errors="coerce")).idmax(axis=1)